In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [6]:
from art.multidevice.vllm import get_llm
import vllm

llm = await get_llm(
    vllm.AsyncEngineArgs(
        model="Qwen/Qwen3-0.6B",
        # pipeline_parallel_size=1,
        # tensor_parallel_size=2,
        # data_parallel_size=2,
        enforce_eager=True,
        generation_config="vllm",
    ),
)

/home/sky/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B/snapshots/e6de91484c29aa9480d55605af694f39b081c455
INFO 06-12 16:57:11 [__init__.py:239] Automatically detected platform cuda.
INFO 06-12 16:57:13 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen3-0.6B', speculative_config=None, tokenizer='Qwen/Qwen3-0.6B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None,

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.95it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.94it/s]



INFO 06-12 16:57:14 [gpu_model_runner.py:1347] Model loading took 1.1201 GiB and 0.377751 seconds
INFO 06-12 16:57:15 [kv_cache_utils.py:634] GPU KV cache size: 1,107,856 tokens
INFO 06-12 16:57:15 [kv_cache_utils.py:637] Maximum concurrency for 40,960 tokens per request: 27.05x
INFO 06-12 16:57:15 [core.py:159] init engine (profile, create kv cache, warmup model) took 0.53 seconds


In [ ]:
import art
from art.local.vllm import openai_server_task

task = await openai_server_task(
    engine=llm,
    config=art.dev.OpenAIServerConfig(
        log_file="./vllm.log",
        server_args=art.dev.ServerArgs(
            api_key="default",
            return_tokens_as_token_ids=True,
            enable_auto_tool_choice=True,
            tool_call_parser="hermes",
        ),
        engine_args=art.dev.EngineArgs(
            model="Qwen/Qwen3-0.6B",
            served_model_name="Qwen/Qwen3-0.6B",
            disable_log_requests=True,
            generation_config="vllm",
        ),
    ),
)

[2025-06-12 16:57:20] INFO _client.py:1740: HTTP Request: GET http://0.0.0.0:8000/v1/models "HTTP/1.1 200 OK"


In [ ]:
await llm.sleep()

In [ ]:
await llm.wake_up()

In [8]:
llm.shutdown()

[rank0]:[W612 16:58:59.025217257 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())
